Sometimes there is an error when trying to create the stim_log using the "make_stim_csv=True" flag. This usually arises if there is some issue with the sweep/sync pulses. Run the code below piece by piece to find the error, it is the same as the repository, but but allows you to interrupt the flow to correct an issue. This will NOT alter the .sync file in any way.

In [1]:
import io
import os
import sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import copy
import time

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib notebook

## Load data

In [5]:
# rec_folder = r'F:\testing\mouse000000\old_test_2023-09-07_14-07-36\experiment1\recording1' # old log style
rec_folder = r'F:\psi_exp\mouse730911\aw_ket_2024-05-02_10-41-41\experiment1\recording1' # new log style
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: electrical and sensory stimulation


### Set parameters and load sync file

In [6]:
VISUAL_STIMULI = ['fullscreen', 'circle', 'natural_scene']

In [7]:
sync_data = exp._load_sync_dataset()

sweep_rising = sync_data.get_edges(keys=('behavior_sweep',), kind='rising', units='seconds')
sweep_falling = sync_data.get_edges(keys=('behavior_sweep',), kind='falling', units='seconds')

vsync_rising = sync_data.get_edges(keys=('behavior_vsync',), kind='rising', units='seconds')
vsync_falling = sync_data.get_edges(keys=('behavior_vsync',), kind='falling', units='seconds')

pd_rising = sync_data.get_edges(keys=('photodiode',), kind='rising', units='seconds')
pd_falling = sync_data.get_edges(keys=('photodiode',), kind='falling', units='seconds')

if len(sweep_rising) == 0:
    between_sweep_inds = np.argwhere(np.diff(vsync_rising) > 10.).flatten()
    sweep_rising = np.concatenate(([vsync_rising[0]-1.], vsync_rising[between_sweep_inds+1]-1.))
    sweep_falling = np.concatenate((vsync_falling[between_sweep_inds]+1., [vsync_falling[-1]+1.]))

### Load experiment log file

In [8]:
explogfile = pd.read_csv(exp.stim_instruction_files['sensory'][0])
explogfile['stim_type'] = [x.replace('\ufeff', '') for x in explogfile['stim_type'].values]
explogfile.head()

,stim_type,parameter,ISI,duration,imtrial.thisRepN,imtrial.thisTrialN,imtrial.thisN,imtrial.thisIndex,thisRow.t,notes,trialStim.started,trialStim.stopped,Unnamed: 12
0,natural_scene,15,2.389600,0.2,0,0,0,0,NaN,NaN,5.120823,5.328218,NaN
1,natural_scene,111,2.250295,0.2,0,1,1,1,NaN,NaN,7.686091,7.897518,NaN
2,natural_scene,25,2.881826,0.2,0,2,2,2,NaN,NaN,10.122258,10.332614,NaN
3,natural_scene,51,2.613754,0.5,0,3,3,3,NaN,NaN,13.196500,13.702123,NaN
4,natural_scene,38,2.598443,1.0,0,4,4,4,NaN,NaN,16.294258,17.305177,NaN


In [9]:
if 'display_stim' in list(explogfile.columns):
    print('This is the old style log')
    explogfile = explogfile.drop(labels='time', axis=1)

    ## Remove gray screen from log ##
    list_stim_type = []
    list_stim_param = []
    for i in range(1, len(explogfile)):
        if explogfile.stim_type.iloc[i-1] == 'gray_screen' and explogfile.stim_type.iloc[i] in VISUAL_STIMULI:
            list_stim_type.append(explogfile.stim_type.iloc[i])
            list_stim_param.append(explogfile.stim_parameter.iloc[i]) # to make sure order matches stim table
    stim_log = pd.DataFrame({
        'stim_type': list_stim_type,
        'parameter': list_stim_param,
    })
    
elif 'ISI' in list(explogfile.columns):
    print('This is the new style log')
    
    stim_log = explogfile.drop(
        labels=[
            'imtrial.thisRepN','imtrial.thisTrialN','imtrial.thisN','imtrial.thisIndex','thisRow.t','notes',
            'trialStim.started','trialStim.stopped','Unnamed: 12'
        ], axis=1
    )
    
else:
    print("Uh oh, tbd_eeg does not understand this log file.")

This is the new style log


### Add photodiode times as onsets/offsets (if available)

In [10]:
if pd_rising.shape[0] == stim_log.shape[0]:
    stim_log['onset'] = pd_rising
    if pd_falling.shape[0] == stim_log.shape[0]:
        stim_log['offset'] = pd_falling
    else:
        print('Number of photodiode offsets does not match number of stimuli; estimating offsets.')
        stim_log['offset'] = stim_log['onset'] + stim_log['duration']
elif vsync_rising.shape[0] == stim_log.shape[0]:
    print('Number of photodiode onsets does not match number of stimuli; using vsync times instead (these are typically 50 ms early).')
    stim_log['onset'] = vsync_rising
    stim_log['offset'] = vsync_falling
else:
    print('Number of photodiode and vsync pulses do not match number of stimuli; manual stim_log creation is needed.')

### Add duration (for old log) and sweep number

In [11]:
if 'duration' not in list(explogfile.columns):
    stim_log['duration'] = stim_log['offset'] - stim_log['onset']
else:
    print('New log already has duration.')

New log already has duration.


In [12]:
stim_log['sweep'] = np.zeros(len(stim_log), dtype=int) - 1
for i in range(len(sweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    stim_log.loc[(stim_log['onset'] > sweep_rising[i]) & (stim_log['onset'] < sweep_falling[i]), 'sweep'] = i

### Check it and save

In [13]:
stim_log.head()

,stim_type,parameter,ISI,duration,onset,offset,sweep
0,natural_scene,15,2.389600,0.2,3047.71965,3047.92055,0
1,natural_scene,111,2.250295,0.2,3050.28845,3050.48937,0
2,natural_scene,25,2.881826,0.2,3052.72383,3052.92473,0
3,natural_scene,51,2.613754,0.5,3055.79305,3056.29423,0
4,natural_scene,38,2.598443,1.0,3058.89565,3059.89724,0


In [34]:
len(stim_log)

360

In [35]:
stim_log[170:190]

,stim_type,parameter,ISI,duration,onset,offset,sweep
170,natural_scene,63,2.717576,0.5,3566.72042,3567.22158,0
171,natural_scene,113,2.810578,0.2,3569.90643,3570.10735,0
172,natural_scene,79,2.373145,1.0,3572.90894,3573.91052,0
173,natural_scene,103,2.072788,0.2,3576.29511,3576.47934,0
174,natural_scene,100,2.208676,0.5,3578.54699,3579.04817,0
175,natural_scene,50,2.601618,0.5,3581.24925,3581.75041,0
176,natural_scene,88,2.897874,0.5,3584.35185,3584.83631,0
177,natural_scene,96,2.830834,1.0,3587.73801,3588.72288,0
178,natural_scene,113,2.090646,1.0,3591.54120,3592.54278,0
179,natural_scene,107,2.042510,0.5,3594.62711,3595.12828,0


In [15]:
all_logs_list = [stim_log] # estim is called estim_log and visual is called stim_log
all_stim_log = pd.concat(all_logs_list, axis=0, sort=False).sort_values(by='onset', axis=0)

In [16]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)

# TESTING

### Old style

In [6]:
old_rec_folder = r'F:\testing\mouse000000\old_test_2023-09-07_14-07-36\experiment1\recording1'
oldexp = EEGexp(old_rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: sensory stimulation


In [8]:
print(oldexp.stim_instruction_files)
print(oldexp.stimulus_log_file)

{'sensory': ['F:\\testing\\mouse000000\\old_test_2023-09-07_14-07-36\\experiment1\\recording1\\2023-09-07_14-09-54_stimulus_log.csv'], 'electrical': [], 'opto': []}
F:\testing\mouse000000\old_test_2023-09-07_14-07-36\experiment1\recording1\sensory_stim_log.csv


In [11]:
osync_data = oldexp._load_sync_dataset()
print(osync_data.line_labels)

['barcodes', 'opto_sync', 'frames', 'behavior_sweep', 'photodiode', 'sweep', '', 'behavior_vsync', 'eyetracking', 'behavior', 'rotA', 'rotB', 'estim_sync', 'estim_sweep', '', '', '', 'opto_sweep', 'opto_trial', '', '', '', '', '', '', '', '', '', '', '', '', '']


### New style

In [7]:
new_rec_folder = r'F:\testing\mouse000000\new_test_2023-09-07_14-07-36\experiment1\recording1'
newexp = EEGexp(new_rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: sensory stimulation


In [9]:
print(newexp.stim_instruction_files)
print(newexp.stimulus_log_file)

{'sensory': ['F:\\testing\\mouse000000\\new_test_2023-09-07_14-07-36\\experiment1\\recording1\\mouse000001_2024-04-04_14h31.16.885_stimulus_log.csv'], 'electrical': [], 'opto': []}
F:\testing\mouse000000\new_test_2023-09-07_14-07-36\experiment1\recording1\sensory_stim_log.csv


In [10]:
nsync_data = newexp._load_sync_dataset()
print(nsync_data.line_labels)

['barcodes', 'opto_sync', 'frames', 'behavior_sweep', 'photodiode', 'sweep', '', 'behavior_vsync', 'eyetracking', 'behavior', 'rotA', 'rotB', 'estim_sync', 'estim_sweep', '', '', '', 'opto_sweep', 'opto_trial', '', '', '', '', '', '', '', '', '', '', '', '', '']


## Compare stimulus_log.csv files

In [12]:
old_log_long = pd.read_csv(oldexp.stim_instruction_files['sensory'][0])
print(len(old_log_long))
old_log_long.head()

122022


,time,frame,display_stim,stim_type,stim_parameter
0,0.004551,0,False,gray_screen,none
1,0.020529,1,False,gray_screen,none
2,0.037279,2,False,gray_screen,none
3,0.054480,3,False,gray_screen,none
4,0.072032,4,False,gray_screen,none


In [13]:
new_log = pd.read_csv(newexp.stim_instruction_files['sensory'][0])
print(len(new_log))
new_log.head()

360


,stim_type,parameter,ISI,duration,imtrial.thisRepN,imtrial.thisTrialN,imtrial.thisN,imtrial.thisIndex,thisRow.t,notes,trialStim.started,trialStim.stopped,Unnamed: 12
0,natural_scene,97,2.412221,1.0,0,0,0,0,NaN,NaN,5.129218,6.143125,NaN
1,natural_scene,59,2.274849,0.2,0,1,1,1,NaN,NaN,8.514028,8.728835,NaN
2,natural_scene,107,2.489749,0.2,0,2,2,2,NaN,NaN,10.973845,11.180777,NaN
3,natural_scene,60,2.895042,1.0,0,3,3,3,NaN,NaN,13.668874,14.683870,NaN
4,natural_scene,71,2.665113,0.5,0,4,4,4,NaN,NaN,17.547715,18.053506,NaN


### How to differentiate?

In [18]:
print(list(old_log_long.columns))
print(list(new_log.columns))

['time', 'frame', 'display_stim', 'stim_type', 'stim_parameter']
['stim_type', 'parameter', 'ISI', 'duration', 'imtrial.thisRepN', 'imtrial.thisTrialN', 'imtrial.thisN', 'imtrial.thisIndex', 'thisRow.t', 'notes', 'trialStim.started', 'trialStim.stopped', 'Unnamed: 12']


In [20]:
testlog = new_log # old_log_long

if 'display_stim' in list(testlog.columns):
    print('this is the old style log')
elif 'ISI' in list(testlog.columns):
    print('this is a new log')
else:
    print("uh oh, I don't understand this log file")

this is a new log


## Clean up and explore the new log

In [21]:
new_log.drop(
    labels=['imtrial.thisRepN','imtrial.thisTrialN','imtrial.thisN','imtrial.thisIndex','thisRow.t','notes','Unnamed: 12'],
    axis=1, inplace=True
)

In [22]:
new_log.head()

,stim_type,parameter,ISI,duration,trialStim.started,trialStim.stopped
0,natural_scene,97,2.412221,1.0,5.129218,6.143125
1,natural_scene,59,2.274849,0.2,8.514028,8.728835
2,natural_scene,107,2.489749,0.2,10.973845,11.180777
3,natural_scene,60,2.895042,1.0,13.668874,14.683870
4,natural_scene,71,2.665113,0.5,17.547715,18.053506


### Get sync data

In [23]:
# behavior_sweep
sweep_rising = nsync_data.get_edges(keys=('behavior_sweep',), kind='rising', units='seconds')
sweep_falling = nsync_data.get_edges(keys=('behavior_sweep',), kind='falling', units='seconds')
# behavior_vsync
vsync_rising = nsync_data.get_edges(keys=('behavior_vsync',), kind='rising', units='seconds')
vsync_falling = nsync_data.get_edges(keys=('behavior_vsync',), kind='falling', units='seconds')
# photodiode
pd_rising = nsync_data.get_edges(keys=('photodiode',), kind='rising', units='seconds')
pd_falling = nsync_data.get_edges(keys=('photodiode',), kind='falling', units='seconds')

In [24]:
print('behavior_sweep info')
print('number of rising/falling: %d/%d' % (len(sweep_rising), len(sweep_falling)))
print('first rising: %f' % sweep_rising[0])
print('first falling: %f' % sweep_falling[0])

behavior_sweep info
number of rising/falling: 2/2
first rising: 12.883450
first falling: 572.053060


In [25]:
print('behavior_vsync info')
print('number of rising/falling: %d/%d' % (len(vsync_rising), len(vsync_falling)))
print('first rising: %f' % vsync_rising[0])
print('first falling: %f' % vsync_falling[0])

behavior_vsync info
number of rising/falling: 360/360
first rising: 18.004600
first falling: 19.007110


In [26]:
print('photodiode info')
print('number of rising/falling: %d/%d' % (len(pd_rising), len(pd_falling)))
print('first rising: %f' % pd_rising[0])
print('first falling: %f' % pd_falling[0])

photodiode info
number of rising/falling: 360/360
first rising: 18.058940
first falling: 19.060410


In [27]:
if pd_rising.shape[0] != new_log.shape[0]:
    print('uh oh, the number of photodiode pulses does not match the number of stimuli')

In [28]:
new_log['pd_on'] = pd_rising
new_log['pd_off'] = pd_falling
new_log['stim_dur'] = new_log['trialStim.stopped'] - new_log['trialStim.started']
new_log['pd_dur'] = new_log['pd_off'] - new_log['pd_on']

In [31]:
new_log.tail()

,stim_type,parameter,ISI,duration,trialStim.started,trialStim.stopped,pd_on,pd_off,stim_dur,pd_dur
355,natural_scene,100,2.401193,1.0,540.824224,541.824753,1270.47339,1271.45820,1.000530,0.98481
356,natural_scene,15,2.615929,0.5,544.208557,544.710467,1273.84288,1274.34392,0.501910,0.50104
357,natural_scene,115,2.342347,0.5,547.308896,547.813232,1276.94547,1277.44654,0.504336,0.50107
358,natural_scene,77,2.100995,0.5,550.152527,550.665620,1279.79786,1280.29893,0.513093,0.50107
359,natural_scene,94,2.324508,0.2,552.735854,552.950886,1282.38334,1282.58416,0.215033,0.20082


In [34]:
print(np.mean(pd_rising - vsync_rising))

0.050326833333329386


In [37]:
print(np.mean(pd_falling - vsync_falling))

0.05333897222222085


Can use the vsync rising signal as a backup for the photodiode. The vsync rising turns on ~50 ms before the image/photodiode actually displays (53 ms for the falling signals).

## Process from stimulus_log.csv

In [39]:
logfile = pd.read_csv(newexp.stim_instruction_files['sensory'][0])
stim_log = logfile.drop(
    labels=[
        'imtrial.thisRepN','imtrial.thisTrialN','imtrial.thisN','imtrial.thisIndex','thisRow.t','notes',
        'trialStim.started','trialStim.stopped','Unnamed: 12'
    ],
    axis=1
)
stim_log.head()

,stim_type,parameter,ISI,duration
0,natural_scene,97,2.412221,1.0
1,natural_scene,59,2.274849,0.2
2,natural_scene,107,2.489749,0.2
3,natural_scene,60,2.895042,1.0
4,natural_scene,71,2.665113,0.5


In [40]:
if pd_rising.shape[0] == stim_log.shape[0]:
    stim_log['onset'] = pd_rising
    if pd_falling.shape[0] == stim_log.shape[0]:
        stim_log['offset'] = pd_falling
    else:
        print('Number of photodiode offsets does not match number of stimuli; estimating offsets.')
        stim_log['offset'] = stim_log['onset'] + stim_log['duration']
elif vsync_rising.shape[0] == stim_log.shape[0]:
    print('Number of photodiode onsets does not match number of stimuli; using vsync times instead (these are typically 50 ms early).')
    stim_log['onset'] = vsync_rising
    stim_log['offset'] = vsync_falling
else:
    print('Number of photodiode and vsync pulses do not match number of stimuli; manual stim_log creation is needed.')
stim_log.head()

,stim_type,parameter,ISI,duration,onset,offset
0,natural_scene,97,2.412221,1.0,18.05894,19.06041
1,natural_scene,59,2.274849,0.2,21.44512,21.64591
2,natural_scene,107,2.489749,0.2,23.89716,24.09797
3,natural_scene,60,2.895042,1.0,26.59942,27.60087
4,natural_scene,71,2.665113,0.5,30.46932,30.97036


In [46]:
stim_log['sweep'] = np.zeros(len(stim_log), dtype=int) - 1
for i in range(len(sweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    stim_log.loc[(stim_log['onset'] > sweep_rising[i]) & (stim_log['onset'] < sweep_falling[i]), 'sweep'] = i

In [48]:
stim_log.tail()

,stim_type,parameter,ISI,duration,onset,offset,sweep
355,natural_scene,100,2.401193,1.0,1270.47339,1271.45820,1
356,natural_scene,15,2.615929,0.5,1273.84288,1274.34392,1
357,natural_scene,115,2.342347,0.5,1276.94547,1277.44654,1
358,natural_scene,77,2.100995,0.5,1279.79786,1280.29893,1
359,natural_scene,94,2.324508,0.2,1282.38334,1282.58416,1


## Sensory stim

This is from NP4 with a digital photodiode.<br>Load behavior_sweep, behavior_sync, and photodiode rising and falling edges.

In [8]:
# behavior_sweep
sweep_rising = sync_data.get_edges(keys=('behavior_sweep',), kind='rising', units='seconds')
sweep_falling = sync_data.get_edges(keys=('behavior_sweep',), kind='falling', units='seconds')
# behavior_vsync
vsync_rising = sync_data.get_edges(keys=('behavior_vsync',), kind='rising', units='seconds')
vsync_falling = sync_data.get_edges(keys=('behavior_vsync',), kind='falling', units='seconds')
# photodiode
pd_rising = sync_data.get_edges(keys=('photodiode',), kind='rising', units='seconds')
pd_falling = sync_data.get_edges(keys=('photodiode',), kind='falling', units='seconds')

In [9]:
if len(sweep_rising) == 0:
    between_sweep_inds = np.argwhere(np.diff(vsync_rising) > 10.).flatten()
    sweep_rising = np.concatenate(([vsync_rising[0]-1.], vsync_rising[between_sweep_inds]+2.))
    sweep_falling = np.concatenate((vsync_rising[between_sweep_inds]+1., [vsync_rising[-1]+1.]))

In [9]:
print('behavior_sweep info')
print('number of rising/falling: %d/%d' % (len(sweep_rising), len(sweep_falling)))
print('first rising: %f' % sweep_rising[0])
print('first falling: %f' % sweep_falling[0])

behavior_sweep info
number of rising/falling: 2/2
first rising: 157.130560
first falling: 763.825500


In [10]:
print('behavior_vsync info')
print('number of rising/falling: %d/%d' % (len(vsync_rising), len(vsync_falling)))
print('first rising: %f' % vsync_rising[0])
print('first falling: %f' % vsync_falling[0])

behavior_vsync info
number of rising/falling: 61794/61794
first rising: 157.131140
first falling: 157.132720


In [11]:
print('photodiode info')
print('number of rising/falling: %d/%d' % (len(pd_rising), len(pd_falling)))
print('first rising: %f' % pd_rising[0])
print('first falling: %f' % pd_falling[0])

photodiode info
number of rising/falling: 240/240
first rising: 211.865310
first falling: 212.097540


Load 'timestamp'_stimulus_log.csv

In [17]:
log_long = pd.read_csv(exp.stim_instruction_files['sensory'][0])
print(len(log_long))
log_long.head()

61338


,time,frame,display_stim,stim_type,stim_parameter
0,0.004293,0,False,gray_screen,none
1,0.017292,1,False,gray_screen,none
2,0.020625,2,False,gray_screen,none
3,0.037331,3,False,gray_screen,none
4,0.054198,4,False,gray_screen,none


Remove "gray" screens from log

In [18]:
## NOW remove reliance on vsync pulses ##
log_long = log_long.drop(labels='time', axis='columns')

## Remove gray screen from log ##
visual_stimuli = ['fullscreen', 'circle', 'natural_scene']
list_stim_type = []
list_stim_param = []
for i in range(1, len(log_long)):
    if log_long.stim_type.iloc[i-1] == 'gray_screen' and log_long.stim_type.iloc[i] in visual_stimuli:
        list_stim_type.append(log_long.stim_type.iloc[i])
        list_stim_param.append(log_long.stim_parameter.iloc[i]) # to make sure order matches stim table
stim_log = pd.DataFrame({
    'stim_type': list_stim_type,
    'parameter': list_stim_param,
})

In [15]:
print(pd_rising[:5])

[191.97224 195.98945 199.92549 204.12917 208.16565]


In [15]:
print(len(pd_rising))

240


In [19]:
## These are not the correct onset/offset times yet
print(len(stim_log))
stim_log.head()

240


,stim_type,parameter
0,natural_scene,33
1,natural_scene,65
2,natural_scene,88
3,natural_scene,51
4,natural_scene,114


Correct onset and offset times using photodiode rising and falling times

In [20]:
stim_log['onset'] = pd_rising
stim_log['offset'] = pd_falling
stim_log['duration'] = stim_log['offset'] - stim_log['onset']
stim_log.head()

,stim_type,parameter,onset,offset,duration
0,natural_scene,33,211.86531,212.09754,0.23223
1,natural_scene,65,215.33486,215.58380,0.24894
2,natural_scene,88,218.33738,219.07006,0.73268
3,natural_scene,51,226.67768,226.85989,0.18221
4,natural_scene,114,229.94706,230.21266,0.26560


In [21]:
stim_log['sweep'] = np.zeros(len(stim_log), dtype=int) - 1
for i in range(len(sweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    stim_log.loc[(stim_log['onset'] > sweep_rising[i]) & (stim_log['onset'] < sweep_falling[i]), 'sweep'] = i

In [22]:
stim_log[0:28]

,stim_type,parameter,onset,offset,duration,sweep
0,natural_scene,33,211.86531,212.09754,0.23223,0
1,natural_scene,65,215.33486,215.58380,0.24894,0
2,natural_scene,88,218.33738,219.07006,0.73268,0
3,natural_scene,51,226.67768,226.85989,0.18221,0
4,natural_scene,114,229.94706,230.21266,0.26560,0
5,natural_scene,103,235.91871,236.15097,0.23226,0
6,natural_scene,117,238.87116,239.15347,0.28231,0
7,natural_scene,116,241.64014,242.38950,0.74936,0
8,natural_scene,82,248.12890,248.86155,0.73265,0
9,natural_scene,40,251.98211,252.74813,0.76602,0


## Combine all logs

In [23]:
all_logs_list = [stim_log] # estim is called estim_log and visual is called stim_log

In [24]:
all_stim_log = pd.concat(all_logs_list, axis=0, sort=False).sort_values(by='onset', axis=0)

In [25]:
all_stim_log.head()

,stim_type,parameter,onset,offset,duration,sweep
0,natural_scene,33,211.86531,212.09754,0.23223,0
1,natural_scene,65,215.33486,215.58380,0.24894,0
2,natural_scene,88,218.33738,219.07006,0.73268,0
3,natural_scene,51,226.67768,226.85989,0.18221,0
4,natural_scene,114,229.94706,230.21266,0.26560,0


## Save the file

In [26]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)

#### Load file to check?